In [1]:
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import os
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import bs4
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import LowercaseFilter
from whoosh import index
from whoosh.filedb.filestore import FileStorage
import re
pd.set_option('display.max_colwidth', 400)

In [2]:
db_df = pd.read_csv('/home/valesco/Datasets/velop/data_broker_urls.csv')
print(len(db_df))
db_df.head()

67


,names,url,search_url,terms_of_service,privacy_policy,opt_out
0,plusaddress,http://plusaddress.com/,http://plusaddress.com/results?cx=partner-pub-5929267120303720%3A1975859289&cof=FORID%3A10&ie=UTF-8&q=[firstname]+[lastname]+[statename]&sa=Search&siteurl=plusaddress.com%2F&ref=&ss=9039j4292519j44,http://plusaddress.com/#terms,http://plusaddress.com/#privacy,null
1,1-800-US-Search,http://www.ussearch.com/consumer/index.jsp,https://www.ussearch.com/search/people/[firstname]/~/[lastname]/[stateabbreviation],http://www.ussearch.com/consumer/commerce/about/terms.jsp,http://www.ussearch.com/consumer/commerce/about/privacy.jsp,"Opt-Out: Yes, http://www.ussearch.com/consumer/commerce/about/privacy.jsp"
2,10 Digits,http://10digits.us,https://whitepages.plus/n/[firstname]_[middleinitial]_[lastname]/[city]_[stateabbreviation]/e454cbeb96819870bcc6aaad5f6fa110,http://10digits.us/terms,http://10digits.us/privacy,null
3,411.com (a WhitePages.com company),http://www.411.com,https://www.411.com/name/[firstname]-[lastname]/[city]-[stateabbreviation],http://www.whitepagescustomers.com/terms-of-service-and-use-agreement/,http://www.whitepagescustomers.com/data-policy/,null
4,4Addresses.com,http://www.4addresses.com,http://4addresses.net/Search.php?q1=[firstname]&q2=[lastname]&t=[statename]&first=0,http://4addresses.com/TermsOfUse.php,http://4addresses.com/PrivacyPolicy.php,null


In [3]:
search_urls = db_df['search_url'].values

cust_info = {
    'firstname': 'Kathleen',
    'middlename': 'Elizabeth',
    'middleinitial': 'E',
    'lastname': 'Petersen',
    'streetaddress': '16175+Feliciana+Ave',
    'streetname': 'Feliciana+Ave',
    'streetnumber': '16175',
    'city': 'Prarieville',
    'statename': 'Louisiana', 
    'stateabbreviation': 'LA',
    'zipcode': '70769',
    'sex': 'Female',
    'phonenumber': ''
    }

document_dict = []
master_ls = []

for url in search_urls:
    if 'http' in url:
        ls = re.findall('\[([A-Za-z0-9_]+)\]', url)
        master_ls.extend(ls)
        
        new_url = url
        for key in ls:
            new_url = new_url.replace('[{}]'.format(key), cust_info[key])
            
        print(new_url)
        browser = webdriver.Chrome()
        browser.get(new_url)

        elements = browser.find_elements_by_xpath('//*')
        
        document = ''
        
        for element in elements:
            try:
                if len(element.text) > 0:
                    document += element.text
            except:
                pass

        time.sleep(1)
        browser.close()
        
        document_name = db_df['names'].loc[db_df['search_url'] == url].values[0]
        
        temp_dict = {}
        temp_dict['name'] = document_name
        temp_dict['doc'] = document
        temp_dict['url'] = new_url
        
        document_dict.append(temp_dict)
        


http://plusaddress.com/results?cx=partner-pub-5929267120303720%3A1975859289&cof=FORID%3A10&ie=UTF-8&q=Kathleen+Petersen+Louisiana&sa=Search&siteurl=plusaddress.com%2F&ref=&ss=9039j4292519j44
https://www.ussearch.com/search/people/Kathleen/~/Petersen/LA
https://whitepages.plus/n/Kathleen_E_Petersen/Prarieville_LA/e454cbeb96819870bcc6aaad5f6fa110
https://www.411.com/name/Kathleen-Petersen/Prarieville-LA
http://4addresses.net/Search.php?q1=Kathleen&q2=Petersen&t=Louisiana&first=0
https://www.addresssearch.com/results.php?type=address&address=16175+Feliciana+Ave&location=Prarieville%2C+LA
http://www.addresses.com/people/Kathleen+Petersen/Prarieville+LA
https://www.advancedbackgroundchecks.com/search/results.aspx?type=&fn=Kathleen&mi=E&ln=Petersen&age=&city=Prarieville&state=Louisiana
http://www.ypstate.com/info/LA/Prarieville/zip/70769 *needstobescraped
https://www.archives.com/search/vital/results?FirstName=Kathleen&LastName=Petersen&Country=United%20States&Region=Louisiana&cert=False#r
h

In [6]:
first_name = cust_info['firstname']
last_name = cust_info['lastname']
city = cust_info['city']
abbrev = cust_info['stateabbreviation']

ddg_url = 'https://duckduckgo.com/?q={}+{}+{}+{}&t=hf&ia=web'.format(first_name, last_name, city, abbrev)

ddg_browser = webdriver.Chrome()
ddg_browser.get(ddg_url)
time.sleep(2)
#get links from search results

for i in range(3):
    ddg_browser.execute_script("window.scrollTo(0, 10000);")
    time.sleep(1)
    
page_links = ddg_browser.find_elements_by_xpath('//a[@class="result__a"]')

link_urls = []
link_name = []

for link in  page_links:
    link_urls.append(link.get_attribute('href'))
    link_name.append(link.get_attribute('href').split('.com')[0] + '.com')

ddg_browser.close()    

ddg_doc_dict = []

for l, link_url in enumerate(link_urls[:50]):
    
    link_browser = webdriver.Chrome()
    link_browser.get(link_url)
    time.sleep(.5)
    
    elements = link_browser.find_elements_by_xpath('//*')
        
    document = ''

    for element in elements:
        try:
            if len(element.text) > 0:
                document += element.text
        except:
            pass

    link_browser.close()

    document_name = link_name[l]

    temp_dict = {}
    temp_dict['name'] = document_name
    temp_dict['doc'] = document
    temp_dict['url'] = link_url

    ddg_doc_dict.append(temp_dict)
    
print(len(page_links), len(ddg_doc_dict))

117 50


In [7]:
final_dict = document_dict + ddg_doc_dict

In [8]:
cust_list = []

for key, value in cust_info.items():
    ls = value.replace('+', ' ').split(' ')
    for word in ls:
        if word not in cust_list and key != 'middleinitial' and word != '':
            cust_list.append(word.lower())

doc_df = pd.DataFrame(columns = ['name', 'url', 'ratio'])            
doc_count = 0 

for doc in final_dict:
    site_name = doc['name']
    site_url = doc['url']
    orig_list = re.sub(r'([^\s\w]|_)+', '', doc['doc'])
    orig_list = orig_list.replace('\n', ' ')
    new_list = orig_list.split(' ')
    final_list = []
    for word in new_list:
        if word != '':
            final_list.append(word.lower())
    
    if len(final_list) > 0:
        word_count = 0

        for cust in cust_list:
            if cust in final_list:
                word_count += final_list.count(cust)

        ratio = word_count / len(final_list)

        doc_df.loc[doc_count] = [site_name, site_url, ratio]
        doc_count += 1
        
doc_df.sort_values('ratio', ascending = False, inplace = True)
doc_df.reset_index(inplace = True, drop = True)
doc_df

,name,url,ratio
0,https://www.bizapedia.com,https://www.bizapedia.com/la/law-office-of-kathleen-e-petersen-llc.html,0.250605
1,https://www.bizapedia.com,https://www.bizapedia.com/people/louisiana/prairieville/kathleen-petersen.html,0.236872
2,Criminal Records,http://www.criminal-records.org/results.php?search_type=background&timeout=20&action=search&fname_req=Kathleen&lname_req=Petersen&middile_initial=&city=Prarieville&approx_age=&state=&imageField.x=62&imageField.y=13,0.180395
3,BlockShopper,https://blockshopper.com/search?q=16175+Feliciana+Ave+Prarieville+LA&type=locations,0.176471
4,State Records (United States State Records),https://staterecords.org/records.php?firstname=Kathleen&lastname=Petersen&city=&state=LA,0.163884
5,Divorce Records,https://www.peoplesmart.com/background-results?FirstName=Kathleen&LastName=Petersen&State=LA&AFF=158&tid=UWP&&utm_source=UWP&utm_medium=affiliate&utm_campaign=UWP&utm_content=UWPFunnel?FirstName=Kathleen&LastName=Petersen&State=LA&AFF=158&tid=UWP&&utm_source=UWP&utm_medium=affiliate&utm_campaign=UWP&utm_content=UWPFunnel,0.159004
6,411.com (a WhitePages.com company),https://www.411.com/name/Kathleen-Petersen/Prarieville-LA,0.158678
7,US Identify,http://www.usidentify.com/search/person.aspx?fn=Kathleen&mn=&ln=Petersen&city=Prarieville&state=LA,0.135814
8,InteliGator,https://www.truthfinder.com/results/?firstName=Kathleen&lastName=Petersen&middleInitial=&city=&state=LA&age=&gender=&theme=&type=&qLocation=false&qRelatives=false&qOver30=false,0.118880
9,http://www.usidentify.com,http://www.usidentify.com/kathleen-petersen/forestgrove+or,0.112738


In [286]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)


result_df = doc_df
result_df['url'] = result_df['url'].apply(lambda x: make_clickable(x))
result_df = result_df[['name', 'url', 'ratio']]
result_df.columns = [['Website', 'Information Link', 'Ratio']]
result_df.style.format(make_clickable)
result_df.to_html('kathleen_result.html', index = False, escape = False)
result_df.to_csv('kathleen_result.csv', index = False)

In [275]:
result_df = pd.read_csv('kathleen_result.csv')
result_df.to_html('kathleen_result.html', index = False, escape = False)

In [279]:
with open('johnny_result.html', 'r') as f:
    doc = f.read()

header = """
<html>
<head>
<meta charset="utf-8">
<style>

.body_bg {{
    }}

.header_image_div {{
    padding-top: 10px;
    width: 100%;
    display: flex;
    align-items: center;
    margin-bottom: 55px;
    }}

.header_image {{
    max-width: 60%;
    max-height: 100%;
    object-fit: cover;
    height: auto;
    display: block;
    margin-left: auto;
    margin-right: auto;
    }}
    
.text_header {{
    font-family: Arial, Helvetica, sans-serif;
    font-size: 18px;
    font-weight: light;
    margin-left:20%;
    margin-right: 20%;
    color: #39425a;
    }}

.dataframe {{
    font-family: Arial, Helvetica, sans-serif;
    font-size: 14px;
    color: #39425a;
    border: 1 px solid #39425a;
    border-collapse: collapse;
    max-width: 70%;
    overflow: scroll;
    text-align: left;
    margin-left: auto;
    margin-right: auto;
    background-color: white;
    margin-bottom: 40px;
    padding: 5px;
    }}

th, td:nth-child(1) {{
    text-align: left;
    font-weight: bold;
    padding: 10px;
    }}
    
th {{
    border: none;
    background-color: #efefef;
    }}
    
td {{
    padding: 8px;
    }}
    
td:nth-child(2) {{
    word-break: break-word;
    padding: 5px;
    }}

    
</style>
</head>
<body class="body_bg">
    <div class="header_image_div">
        <img src="https://imgur.com/WBR2bNH.png" class="header_image"></img>
    </div>
    <div class="text_header">
        <H4> {}, below are the results of our web scan for your information. Thanks for your business.
        </H4>
    </div>
""".format(cust_info['firstname'])

footer = """

</body>
</html>"""

doc = header + doc + footer
now = datetime.now()
now_str = datetime.strftime(now, '%m_%d_%Y %H:%M')

with open('johnny_result_{}.html'.format(now_str), 'w') as outfile:
    outfile.write(doc)
    
    

In [281]:
sender = 'support@veloptechnologies.com'
receiver = 'jm10217@gmail.com'

def send_email(message, sender, receiver):
    now_string = time.strftime('%m_%d_%Y_%H:%M')
    msg = MIMEMultipart('alternative')
    msg['Subject'] = 'Your Web Scrape Results'
    msg['From'] = sender
    msg['To'] = receiver
    body = MIMEText(doc, 'html')
    msg.attach(body)
    
    gmail_user = 'veloptechnologies@gmail.com'
    gmail_pwd = '!Z3c5b7m9.'

    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.login(gmail_user, gmail_pwd)
    server.sendmail(sender, receiver, msg.as_string())
    server.close()
    
    #file = '/home/valesco/Datasets/crypto/reddit_scrapes/reddit_scrap' + now_string + '.txt'
    #with open(file, 'w') as f:
        #f.write("\n".join(map(lambda x: str(x), message)))
        #f.close()
    print('email sent ' + now_string)
    
send_email(doc, sender, receiver)

email sent 03_05_2018_12:37


In [265]:
result_df.to_csv('kathleen_result.csv', index = False)

In [297]:
0 % 100 == 0

True